In [12]:
import requests
from pandas import DataFrame
from concurrent import futures

In [13]:
with open('서울교통공사_역주소 및 전화번호_20250318.csv','r',encoding='euc-kr') as f:
    csv_list=f.readlines()
print(csv_list[:5])

['연번,역번호,호선,역명,역전화번호,도로명주소,지번주소\n', '1,150,1,서울,02-6110-1331,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선)\n', '2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선)\n', '3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선)\n', '4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선)\n']


In [14]:
url='https://api.vworld.kr/req/address'
params = {
    "service": "address",
    "request": "getcoord",
    "crs": "epsg:4326",
    "address": "판교로 242",
    "format": "json",
    "type": "road",
    "key": "A2B16D11-901E-38A8-AE1E-B54CD38497E8"
}

In [15]:
with requests.Session() as session:
    session.headers.update({
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })
    r = session.get(url, params=params)
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    print(r)

<Response [200]>


In [16]:
mydict = r.json()
mydict

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getcoord',
   'time': '19(ms)'},
  'status': 'OK',
  'input': {'type': 'road', 'address': '판교로 242'},
  'refined': {'text': '경기도 성남시 분당구 판교로 242 (삼평동)',
   'structure': {'level0': '대한민국',
    'level1': '경기도',
    'level2': '성남시 분당구',
    'level3': '삼평동',
    'level4L': '판교로',
    'level4LC': '',
    'level4A': '삼평동',
    'level4AC': '4113565500',
    'level5': '242',
    'detail': ''}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '127.101313354', 'y': '37.402352535'}}}}

In [17]:
point = mydict['response']['result']["point"]
print(point['x'], point['y'])

127.101313354 37.402352535


In [18]:
def get_point(addr, type="road"):
    url = "https://api.vworld.kr/req/address"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": addr,
        "format": "json",
        "type": type,
        "key": "A2B16D11-901E-38A8-AE1E-B54CD38497E8"
    }

    with requests.Session() as session:
        session.headers.update({
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
        })
        r = session.get(url, params=params)
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    
    mydict = r.json()
    point = mydict['response']['result']["point"]
    return (point['x'],point['y'])

In [9]:
# 동기식 처리 (오래걸림)
size=len(csv_list)
resultset=[]
for i, v in enumerate(csv_list[1:]):                        # 주석 제외할 때에 [1:] 사용
    #print('%d/%d 진행중...'%(i+1,size))
    items=v.strip().split(',')

    try:
        lat, lon=get_point(items[5])                        # 도로명 주소로 우선 검색
    except Exception as e:
        try:
            lat, lon=get_point(items[6],type='parcel')      # 도로명 주소가 안되면 지번 주소로 검색 (사이트에서 지정한 지번 주소 타입은 parcel이다)
        except Exception as e2:
            lat=None
            lon=None
    
    items.append(lat)
    items.append(lon)
    resultset.append(items)                                 # items라는 리스트를 원소로 넣어버리기

#resultset

In [19]:
# 비동기식 처리 (효율적)
size=len(csv_list)
processes=[]
resultset=[]
with futures.ThreadPoolExecutor(max_workers=30) as executor:
    for i, v in enumerate(csv_list[1:]):
        #print('%d/%d 진행중...'%(i+1,size))
        items=v.strip().split(',')
        pro=executor.submit(get_point, items[5])
        processes.append(pro)

    for i, p in enumerate(processes):
        try:
            lat, lon=p.result()
        except Exception as e:
            try:
                lat, lon=get_point(items[6],type='parcel')
            except Exception as e2:
                lat=None
                lon=None
        
        items=csv_list[i].strip().split(',')
        items.append(lat)
        items.append(lon)
        resultset.append(items)
    
#resultset

In [24]:
new_resultset=[]

for r in resultset[1:]:
    item_dict={
        '연번':r[0],
        '역번호':r[1],
        '호선':r[2],
        '역명':r[3],
        '역전화번호':r[4],
        '도로명주소':r[5],
        '지번주소':r[6],
        '위도':r[8],
        '경도':r[7]
    }

    new_resultset.append(item_dict)

df=DataFrame(new_resultset)
df.to_excel('지하철역.xlsx')
df

,연번,역번호,호선,역명,역전화번호,도로명주소,지번주소,위도,경도
0,1,150,1,서울,02-6110-1331,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선),37.566700969,126.978346780
1,2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선),37.573411520,127.022379505
2,3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선),37.573411520,127.022379505
3,4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선),37.573411520,127.022379505
4,5,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하216(종로5가),서울특별시 종로구 종로5가 82-1 종로5가역(1호선),37.573411520,127.022379505
...,...,...,...,...,...,...,...,...,...
284,285,4134,9,송파나루,02-2656-0934,서울특별시 송파구 백제고분로 지하446(방이동),서울특별시 송파구 방이동 2 송파나루역(9호선),37.512011982,127.118345402
285,286,4135,9,한성백제,02-2656-0935,서울특별시 송파구 위례성대로 지하29(방이동),서울특별시 송파구 방이동 88-17 한성백제역(9호선),37.515450096,127.134326509
286,287,4136,9,올림픽공원(한국체대),02-2656-0936,서울특별시 송파구 양재대로 지하1233(방이동),서울특별시 송파구 방이동 89-28 올림픽공원역(9호선),37.524208018,127.132180703
287,288,4137,9,둔촌오륜,02-2656-0937,서울특별시 강동구 강동대로 지하303(둔촌동),서울특별시 강동구 둔촌동 227-7 둔촌오륜역(9호선),37.517427151,127.144364997
